In [1]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 31.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
!pip install gradio transformers torch openai-whisper gTTS librosa soundfile spacy pydub tensorflow tensorflow_hub speechbrain torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 14.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.8/824.8 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 10.8 MB/s et

In [3]:
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/audioset/yamnet/yamnet_class_map.csv
!pip install openai-whisper
!pip install git+https://github.com/openai/whisper.git
!pip install git+https://github.com/speechbrain/speechbrain.git@develop
# !pip install torch

--2024-11-28 19:33:14--  https://raw.githubusercontent.com/tensorflow/models/master/research/audioset/yamnet/yamnet_class_map.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14096 (14K) [text/plain]
Saving to: ‘yamnet_class_map.csv’

yamnet_class_map.cs 100%[===================>]  13.77K  --.-KB/s    in 0s      

2024-11-28 19:33:14 (104 MB/s) - ‘yamnet_class_map.csv’ saved [14096/14096]

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-0ab0zxy_
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-0ab0zxy_
  Resolved https://github.com/openai/whisper.git to commit fc5ded7d9045c693692f13853857c3f8baea3a7b
  Installing build dependencies ... done
  Getting requirements to build

In [4]:
import whisper
whisper_model = whisper.load_model("base")

100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 91.4MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [5]:
emotion_mapping = {
    "LABEL_0": "sadness",
    "LABEL_1": "happy",
    "LABEL_2": "love",
    "LABEL_3": "anger",
    "LABEL_4": "fear",
    "LABEL_5": "surprise"
}

In [ ]:
# THIS IS WORKING OKAY, GENRE CLASSIFICATION IS GIVING FALSE ERRORS!!!

import whisper
from transformers import pipeline
from gtts import gTTS
import gradio as gr
import re
import torch
import librosa
import soundfile as sf
import torchaudio
import spacy
import numpy as np
from pydub import AudioSegment

asr_model = whisper.load_model("base") # speech recog
# nlp_model = pipeline("text2text-generation", model="google/flan-t5-large")
# music_genre_classifier = pipeline("audio-classification", model="SeyedAli/Musical-genres-Classification-Hubert-V1")
# summarizer = pipeline("summarization", model="t5-base")
# sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
# emotion_recognizer = pipeline("text-classification", model="mrm8488/t5-base-finetuned-emotion")
nlp_model = pipeline("text2text-generation", model="google/flan-t5-large", device=0)
music_genre_classifier = pipeline("audio-classification", model="SeyedAli/Musical-genres-Classification-Hubert-V1", device=0)
summarizer = pipeline("summarization", model="t5-base", device=0)
sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english", device=0)
emotion_recognizer = pipeline("text-classification", model="mrm8488/t5-base-finetuned-emotion", device=0)
nlp = spacy.load("en_core_web_sm")

def convert_to_wav(file_path):
    if not file_path.endswith(".wav"):
        audio = AudioSegment.from_file(file_path)
        wav_path = "converted_temp.wav"
        audio.export(wav_path, format="wav")
        return wav_path
    return file_path

def split_audio_into_chunks(file_path, chunk_length=60):
    audio = AudioSegment.from_wav(file_path)
    chunk_length_ms = chunk_length * 1000
    chunks = [audio[i:i + chunk_length_ms] for i in range(0, len(audio), chunk_length_ms)]
    return chunks

def transcribe_chunk(chunk):
    temp_path = "temp_chunk.wav"
    chunk.export(temp_path, format="wav")
    result = asr_model.transcribe(temp_path)
    return result['text']

def get_combined_transcription(audio_file, chunk_length=30):
    file_path = convert_to_wav(audio_file)
    chunks = split_audio_into_chunks(file_path, chunk_length=chunk_length)
    combined_transcription = ""
    for chunk in chunks:
        transcription = transcribe_chunk(chunk)
        combined_transcription += transcription + " "
    return combined_transcription.strip()

def text_to_audio(text):
    tts = gTTS(text=text, lang='en')
    tts.save("response.mp3")
    return "response.mp3"

def classify_text_emotion(text):
    result = emotion_recognizer(text)
    raw_emotion = result[0]["label"]
    emotion = emotion_mapping.get(raw_emotion, "unknown emotion")
    return emotion

def classify_music_genre(audio_file):
    result = music_genre_classifier(audio_file)
    # print(result)
    try:
        genre_label = result[0].get("label", "Unknown Genre")
        confidence = round(result[0].get("score", 0), 3)
    except (IndexError, KeyError):
        genre_label = "Unknown Genre"
        confidence = 0.0
    return [{"Genre": genre_label, "Confidence": confidence}]

def generate_descriptive_insights(transcription, audio_emotion):
    truncated_transcription = transcription[:2048]

    summary = summarizer(
        f"Summarize the following text by focusing on the main ideas and overall mood: {truncated_transcription}",
        max_length=650,
        min_length=25,
        do_sample=False
    )
    summarized_text = summary[0]['summary_text']

    sentiment_result = sentiment_analyzer(truncated_transcription)
    mood = sentiment_result[0]['label'].lower()

    mood_description = (
        f"The speaker's mood is likely {mood}. "
        f"Emotionally, the analysis suggests {audio_emotion}. "
        f"Key themes include: {summarized_text}"
    )

    return mood_description

def process_audio_and_text(input_audio=None, input_text=None):
    transcription = get_combined_transcription(input_audio) if input_audio else input_text
    response_text = transcription
    audio_text = transcription  # Initialize audio_text to avoid UnboundLocalError

    if len(transcription) > 512:
        transcription = transcription[:512]

    if input_text:
        prompt = input_text.lower()

        if "genre" in prompt or "type of song" in prompt:
            genre_result = classify_music_genre(input_audio)

            # Ensure genre_result is not empty before accessing its contents
            if genre_result and "label" in genre_result[0]:
                genre = genre_result[0]['label']
                confidence = genre_result[0]['Confidence']
                response_text = {"Genre Prediction": {"Genre": genre, "Confidence": confidence}}
                audio_text = f"The genre is {genre} with a confidence of {confidence}."
            else:
                response_text = {"Genre Prediction": "Unknown Genre"}
                audio_text = "The genre could not be determined. Maybe input a clearer recording or relevant text input."

        elif "mood" in prompt or "feeling" in prompt:
            genre_result = classify_music_genre(input_audio)
            genre = genre_result[0]['label'] if genre_result and "label" in genre_result[0] else "Unknown Genre"

            audio_emotion = classify_text_emotion(transcription)
            sentiment_result = sentiment_analyzer(transcription)
            sentiment_mood = sentiment_result[0]['label'].lower()

            response_text = {
                "Mood": f"{audio_emotion} with a {sentiment_mood} tone, fitting the genre '{genre}'"
            }
            audio_text = f"The mood is {audio_emotion}, giving a {sentiment_mood} feeling in the genre {genre}."

        elif "insight" in prompt or "saying" in prompt or "summary" in prompt:
            audio_emotion = classify_text_emotion(transcription)
            descriptive_insights = generate_descriptive_insights(transcription, audio_emotion)
            response_text = {"Descriptive Insight": descriptive_insights}
            audio_text = descriptive_insights

        elif re.search(r"\d+ [+\-*/] \d+", transcription):
            try:
                answer = str(eval(transcription))
                response_text = f"The answer is {answer}."
                audio_text = response_text
            except:
                response_text = "I'm sorry, I couldn't calculate that."
                audio_text = response_text

        else:
            response_text = nlp_model(f"Provide a very detailed and comprehensive explanation for the following topic: {transcription}.",
                                      max_length=550, do_sample=False, top_k=50, top_p=0.9, temperature=0.4, repetition_penalty=1.2)[0]['generated_text']
            audio_text = response_text

    audio_response = text_to_audio(audio_text)
    return transcription, response_text, audio_response

# Gradio Interface
with gr.Blocks(theme='earneleh/paris') as demo:
    with gr.Row():
        audio_input = gr.Audio(type="filepath", label="Upload Audio File")
        optional_text = gr.Textbox(lines=2, label="Optional Text Input (e.g., 'What is the mood?' or 'What is the genre?' or 'Give me insights')")

    with gr.Row():
        audio_transcription = gr.Textbox(label="Transcription", placeholder="Audio transcription will appear here.")
        output_insights = gr.JSON(label="Derived Insights or Answer")
        output_audio = gr.Audio(label="Audio Response")

    audio_submit = gr.Button("Process Audio and Text")
    audio_submit.click(fn=process_audio_and_text, inputs=[audio_input, optional_text], outputs=[audio_transcription, output_insights, output_audio])

demo.launch(debug=True)

/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


themes/theme_schema@0.0.1.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c88a0e868a9f61ed0c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 2026, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1832, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
  File "/usr/local/lib/python3.10/dist-packages/gradio/components/json_component.py", line 115, in postprocess
    return JsonData(orjson.loads(value))
orjson.JSONDecodeError: unexpected character: line 1 column 1 (char 0)
